# Artigo 2: Deploy do Modelo de Aprendizado de Máquina Trabalhado No Artigo "Identificar o que é jogo de tiro e jogo de futebol" 

## Autor
- **Levi de Oliveira Queiroz**
- **Matrícula: 170108341**
- **GitHub: LeviQ27**

## Objetivo 

O objetivo do presente trabalho é demonstrar a criação de um exemplo de um aplicativo visual de computação do modelo de aprendizado de máquina produzido no artigo 1, com algumas alterações em sua linha de código. Demonstrando também o Deploy do modelo em um aplicativo online. Isso aplicando os conhecimentos adquiridos na lição 2 "From Model to Production".

## Inferência

Como parte do escopo do artigo, segue as referências do curso de Aprendizagem de Máquina do *fast.ai*, a referência do aplicativo entregue no site *HuggingFace* e o link do notebook separado para testar o modelo criado neste presente trabalho.

- Curso *fast.ai* Capítulo 2: https://course.fast.ai/Lessons/lesson2.html

- Aplicativo rodando no *HuggingFace*: https://huggingface.co/spaces/L27Queiroz/Lesson2

- Jupyter Notebook usado para gerar o aplicativo Python: [ImageClassifier.ipynb](ImageClassifier.ipynb)

## Desenvolvimento 

O projeto estará compilando uma diferença entre jogo de tiro, jogo de futebol e, de forma adicional, jogo de fantasia. E tem-se uma coleção de imagens na internet para poder processar os dados. Assim, usando a função ```search_images_ddg``` da biblioteca ```duckduckgo_search```, foi adicionado algumas url's de imagen de jogo de fantasia. 

Contudo, para ser iniciado a busca pela url, foi instalado a do ```fastbook``` seguindo a diretriz do livro criado pelo seu Howard:

In [ ]:
%pip install -Uqq fastbook

In [ ]:
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *

Em seguida, é definido a função ```search_images_ddg```:

In [ ]:
search_images_ddg

Em seguida, a função ```search_images_ddg``` vai baixar algumas url's de imagem de jogo de fantasia para a variável ```ims```:

In [ ]:
ims = search_images_ddg('fantasy game')
len(ims)

Certo, a funçao ```search_images_ddg``` baixou ~200 url's de imagens de jogo de fantasia. Em seguida, é verificado uma imagem das url's baixadas, para verificar se a função funcionou como foi ordenada:

In [ ]:
dest = 'images/fantasygame.jpg'
download_url(ims[0], dest, show_progress=False)

Agora, depois de selecionado uma url, é verificado a imagem:

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

> Note que é criado uma varial que recebe a visualização da imagem e essa imagem através do ```to_thumb``` é redimensionada.

Observado que a função retornou dentro do esperado, é usando a função ```download_images``` para trazer as imagens, através de suas url's, necessárias para a continuação do presente trabalho, em que será puxado as url"s de jogos de tiro, futebol e fantasia:

In [ ]:
game_types = 'shooting game','soccer game','fantasy game'
path = Path('games')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in game_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} videogame')
        download_images(dest, urls=results)

> Note que as imagens ficaram em pastas separadas

Assim como foi observado, foi criado a pasta ```games``` em que há as três subpastas com as imagens:

In [ ]:
fns = get_image_files(path)
fns

> É utilizado a função ```get_image_files``` para olhar o caminho das imagens baixadas nos arquivos

Como é observado, é possível encontrar o caminho dos dados que são necessários para o andamento do projeto. Agora, é de conhecimento comum que quando se baixa imagens da internet algumas imagens acabam ficando corrompidas. Então para verificar e remover as imagens que estão corrompidas, é usado a função ```verify_images``` para verificar a condição das imagens, ```unlink``` para remover as imagens e como essa função retorna um objeto, isso inclui o método ```map``` que será usado para encontrar a localização das figuras:

In [ ]:
failed = verify_images(fns)
failed

> Note que das ~539 imagens baixadas, ~30 delas deram resultado negativo, ou seja, estão corrompidas.

In [ ]:
failed.map(Path.unlink);

Com o processo de filtro de imagens corrompidas completo, é iniciado o processo de construção do treino para o modelo de aprendizagem de máquina. Para isso é usado a classe ```DataLoader```, essa classe permite que dados sejam transformados em treino e validação. É uma classe importante pois ela monta os dados para o modelo de aprendizagem de máquina.

E para montar e customizar o ```DataLoader``` é utilizado um sistema flexivel do fastai chamado *data block API*:

In [ ]:
games = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y = parent_label,
    item_tfms=Resize(128)
)

Com o ```DataBlock``` montado, basta montar o ```DataLoader```:

In [ ]:
dls = games.dataloaders(path)

Para mostrar se o ```DataLoader``` funcionou, é usado o método ```show_batch``` que vai apresentar um número *n* de imagens pedidas:

In [ ]:
dls.valid.show_batch(max_n=12, nrows=3)

No capítulo 2 do livro do seu Howard é explicado varias formas de se organizar e customizar o ```Dataloader```, porém não será apresentado no presente trabalho.

## Treinando o Modelo e o Usando para Limpar Seus Dados

Certo, como é possível ver nas imagens anteriores, algumas imagens não correspondem com o que é esperado. Sendo assim, existe uma forma de treinar e corrigir o modelo de aprendizagem de máquina.

Iniciando pelo treino:

In [ ]:
games = games.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
)

dls = games.dataloaders(path)

Agora, é criado o ```Learner``` e aprimorando no caminho usual:

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Com o treino completo, é bom visualizar a taixa de erro, pois o modelo pode errar achando que uma imagem do jogo é de um tipo sendo que não é. Assim, é utilizado uma *matriz de confusão* para visualizar os erros:

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Foi observado quem em alguns casos o modelo ficou confuso sobre certas imagens. E para observamos as *perdas* que teve nessa confusão, é usado o método ```plot_top_losses``` que vai apresentar as maiores *perdas*, sendo por erro mesmo ou por desconfiança do modelo.

In [ ]:
interp.plot_top_losses(5, nrows=5)

> No caso, todos os 5 *top_losses* foram por erros

Agora, o *fastai* suporta um GUI para limpeza de dados chamado ```ImageClassifierCleaner``` que permite a escolha da categoria e de construção treino x validação. Usando essa função: 

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

Assim para aplicar as mudanças, será utilizado:

In [ ]:
shooting = 'games/shooting game'
soccer = 'games/soccer game'
fantasy = 'games/fantasy game'

for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,shooting in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/shooting)
for idx,soccer in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/soccer)
for idx,fantasy in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/fantasy)

Depois de treinado o modelo, seguindo o livro do seu Howard, será feito o deploy do modelo em uma aplicativo online.

## Transformando seu Modelo em um Aplicativo OnLine

Para isso, foi exportado o modelo feito nos tópicos anteriores:

In [ ]:
learn.export()

E é verificado a existência de um arquivo *.pkl*:

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

Verificado a existência do artefato, foi criado outro repositório no *HuggingFace* para o deploy do aplicativo. Foi construido um outro notebook onde foi importando o modelo treinado.

Logo abaixo está localizado o notebook criado para produzir o aplicativo Python para rodar no *HuggingFace*:

[Image Classifier](ImageClassifier.ipynb)

Segue o link do repositório do aplicativo no *HuggingFace*:

[Lesson 2 - *HuggingFace* Space Model](https://huggingface.co/spaces/L27Queiroz/Lesson2)

## Conclusão

Foi uma experiência diferente usar o *HuggingFace* para fazer deploy de aplicação. Durante o projeto tive também algumas experiências negativas como conflito de versões de bibliotecas do python, a falta de explicação de *build* no ambiente do *HuggingFace* utilizando o arquivo *requirements.txt*, conflito não resolvido com a biblioteca nbdev.export com o atributo notebook2script, em que ainda não foi resolvido para ter a autogeração do artefato *app.py* direto do notebook. Ainda assim, tendo problema na resolução final do aplicativo no HuggingFace, em que não está acertando o tipo de jogo através dos exemplos inseridos nele.